# Initial preparations

We need to first install all the necessary python packages and download the data we will use in this tutorial. Before we do this, you should create a python environment specifically for processing neuroimaging data with python. [Anaconda](https://www.anaconda.com/products/distribution) is an easy tool to create and manage python environments. Install Anaconda (if don't already have it installed) and create a new environment with the latest version python. Then, with this environment activated (or selected in VS Code), move on to install the necessary python packages.

## Python packages
All the packages we need to install are included in the requirements.txt file in this repository. We can install these packages with a call to pip (note: in a jupyter notebook, you can make calls to the terminal with a %, this command could also be run within a terminal):

In [3]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Download the tutorial data
We are using data from the Pinel Localizer task which includes a 5-minute functional localizer for a few basic cognitive processes (visual perception, finger tapping, langauge, math). The [original paper](https://bmcneurosci.biomedcentral.com/articles/10.1186/1471-2202-8-91) and [OSF website](https://osf.io/vhtf6/files/) include all the details. The full dataset includes 94 subject and is big (~42Gb), we will only grab a subset of the data (~6Gb). 

The data is available through a DataLad instance, which thankfully has a python API. DataLad was included in the requirements.txt file above, so was installed with the rest of the python packages. But, for DataLad to work properly, we also need to install git-annex. See [https://git-annex.branchable.com/install/] for installations instructions (on MacOS, use Homebrew to install: `brew install git-annex`). **Install git-annex before proceeding!**

Once git-annex is installed, we can download the data. NOTE: In the code below, make sure to update `localizer_path` to point to a directory on your computer where the data should be downloaded.

In [8]:
import os
import glob
import datalad.api as dl
import pandas as pd

# update this path to a local directory on your computer!
localizer_path = '/Users/michael/Dropbox/work/data/dartbrains/data/localizer'

# clone the datalad repository and create a local dataset instance (this will take several minutes!)
dl.clone(source='https://gin.g-node.org/ljchang/Localizer',path=localizer_path)
ds = dl.Dataset(localizer_path)

Cloning the dataset to your local computer only provides links to the file structure, we still need to actually download the data. The get calls below will take some time to complete (30 mins!), so get it started and go grab a coffee. 

In [14]:
# download the experiment metadata
result = ds.get(glob.glob(os.path.join(localizer_path,'*.json')))
result = ds.get(glob.glob(os.path.join(localizer_path,'*.tsv')))
result = ds.get(glob.glob(os.path.join(localizer_path, 'phenotype')))
# download the first 5 subjects fmriprep'd data
file_list = glob.glob(os.path.join(localizer_path,'*','fmriprep','sub*'))
file_list.sort()
for f in file_list[:10]:
    result = ds.get(f)

action summary:
  get (notneeded: 3)


In [12]:
file_list

['/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S01',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S01.html',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S02',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S02.html',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S03',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S03.html',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S04',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S04.html',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S05',
 '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/derivatives/fmriprep/sub-S05.html',
 '/Users/michael/Dropbox/work/data/dartbrains/data/loca

In [13]:
ds.status(annex='all')

1794 annex'd files (1.1 GB/42.1 GB present/total size)
nothing to save, working tree clean


[{'type': 'file',
  'gitshasum': '39d13d9f0ba20d7bcd64b0960de52c1588168367',
  'prev_gitshasum': '39d13d9f0ba20d7bcd64b0960de52c1588168367',
  'state': 'clean',
  'backend': 'MD5E',
  'bytesize': 22532,
  'hashdirlower': '64f/7d7/',
  'hashdirmixed': 'G4/ZF/',
  'humansize': '22.53 kB',
  'key': 'MD5E-s22532--fb0eb53efee7d0081bd052f143b1bc61',
  'keyname': 'fb0eb53efee7d0081bd052f143b1bc61',
  'mtime': 'unknown',
  'has_content': False,
  'path': '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/.DS_Store',
  'parentds': '/Users/michael/Dropbox/work/data/dartbrains/data/localizer',
  'status': 'ok',
  'refds': '/Users/michael/Dropbox/work/data/dartbrains/data/localizer',
  'action': 'status'},
 {'type': 'file',
  'gitshasum': 'b540820107ca5718dc0f196a08edf3729239bfef',
  'bytesize': 125,
  'prev_gitshasum': 'b540820107ca5718dc0f196a08edf3729239bfef',
  'state': 'clean',
  'path': '/Users/michael/Dropbox/work/data/dartbrains/data/localizer/.datalad/.gitattributes',
  'parentds